<div style="text-align:center;">
  <b>Национальный исследовательский университет ИТМО</b><br>
  Факультет информационных технологий и программирования<br>
  Кафедра Компьютерных Технологий
</div>


---

# **Методы оптимизации**  
### Лабораторная работа №2 
### Продвинутые методы

<div style="text-align:right; font-size:12px">
Выполнили:<br>
Алфёров Кирилл М3232<br>
Салов Егор М3232
</div>





### IMPORTS

In [ ]:
import numpy as np
import sympy
from numpy import ndarray
from scipy.optimize._linesearch import line_search
from scipy.optimize import minimize
from sympy import symbols, lambdify, PrecisionExhausted

from source.BackTrace import BackTrace
from source.Function import Function
from source.graphics import *
from source.one_dimensional_search.Dichotomy import dichotomy
from source.one_dimensional_search.GoldenSectionSearch import golden_section_search

x, y = symbols('x y')

#### Функция №1


$$
f(x, y) = x^2 + y^2 - 1
$$
---
---

#### Функция №2

$$
f(x, y) = sin(x) * cos(y) + x^2 + y^2
$$
---
---

## Метод Ньютона с выбором шага и одномерным поиском

### 1.1. Реализация метода Ньютона с выбором шага

Метод Ньютона — это итерационный метод оптимизации, в котором используются градиент и гессиан. Точка обновляется по формуле:

$$
x_{k+1} = x_k + \alpha_k \cdot p_k, \quad \text{где } p_k = -[\nabla^2 f(x_k)]^{-1} \nabla f(x_k)
$$

<p>
Длина шага <code>&alpha;<sub>k</sub></code> не фиксирована, а подбирается с помощью одномерного поиска — например, методом дихотомии или золотого сечения — для минимизации вдоль направления <code>p_k</code>:
</p>

$$
\phi(\alpha) = f(x_k + \alpha \cdot p_k)
$$

#### Градиент и гессиан

Для функции  
$$f(x_1, x_2, \dots, x_n)$$  
градиент и гессиан определяются как:

$$
\nabla f(x) = \left( \frac{\partial f}{\partial x_1}, \dots, \frac{\partial f}{\partial x_n} \right), \quad 
\nabla^2 f(x) = \left[ \frac{\partial^2 f}{\partial x_i \partial x_j} \right]_{i,j}
$$

---

#### **Преимущества:**

- Быстрая сходимость 
- Точное направление движения за счёт учёта гессиана
- Шаг подбирается автоматически, не требует ручного выбора

---

#### **Недостатки:**

- Требует вычисления и инверсии гессиана
- Не работает при вырожденном или неопределённом гессиане
- Более сложная реализация, чем у градиентного спуска

Метод Ньютона с одномерным поиском хорошо работает на гладких, выпуклых функциях, но требует аккуратного обращения с матрицей вторых производных.


функция для сравнения x^10 + y^2

---

### Анализ поведения на функции №1

#### Константный шаг

In [ ]:
func = Function(-x ** 2 - y ** 2 + 10)
try:
    BackTrace(func).set_constant_step(constant_step=0.3).set_create_graphic().start_back_trace()
except Exception:
    pass

---

#### Кусочно-постоянный шаг

In [ ]:
try:
    BackTrace(func).set_piecewise_constant(start_step=0.5).set_create_graphic().start_back_trace()
except Exception:
    pass

---

#### Экспоненциальное затухание

In [ ]:
try:
    BackTrace(func).set_exponential_decay(exponential_decay=0.5,
                                          start_step=1).set_create_graphic().set_full_output().start_back_trace()
except Exception:
    pass


---

#### Реализация с методом одномерного поиска: метод золотого сечения

In [ ]:
try:
    BackTrace(func).set_golden_section_step(0.7, 0.7).set_create_graphic().set_full_output().start_back_trace()
except Exception:
    pass


---

#### Реализация с методом одномерного поиска: метод дихотомии

In [ ]:
try:
    BackTrace(func).set_dichotomy_step(0.7, 0.7).set_create_graphic().set_full_output().start_back_trace()
except Exception:
    pass


---

### Анализ поведения на функции №2

#### Константный шаг

In [ ]:
func = Function(sympy.sin(x) * sympy.cos(y) + x ** 2 + y ** 2)
BackTrace(func).set_constant_step(constant_step=0.3).set_create_graphic().start_back_trace()

---

#### Кусочно-постоянный шаг

In [ ]:
BackTrace(func).set_piecewise_constant(start_step=0.5).set_create_graphic().start_back_trace()

---

#### Экспоненциальное затухание

In [ ]:
BackTrace(func).set_exponential_decay(exponential_decay=0.05,
                                      start_step=1).set_create_graphic().set_full_output().start_back_trace()

---

#### Реализация с методом одномерного поиска: метод золотого сечения

In [ ]:
BackTrace(func).set_golden_section_step(0.7, 0.7).set_create_graphic().set_full_output().start_back_trace()


---

#### Реализация с методом одномерного поиска: метод дихотомии

In [ ]:
BackTrace(func).set_dichotomy_step(0.7, 0.7).set_create_graphic().set_full_output().start_back_trace()


---

## Методы из scipy.optimize: Newton-CG и квазиньютоновские.

## Использование `scipy.optimize` для численной оптимизации

Библиотека `scipy.optimize` предоставляет готовые реализации различных методов оптимизации, которые мы проанализировали и использовали

В этой работе использовались три метода:

---

### BFGS (`method="BFGS"`)

Это стандартная реализация квазиньютоновского метода BFGS:

- приближение обратного Гессиана обновляется на каждой итерации
- требуется только градиент (Гессиан не нужен)


```python
from scipy.optimize import minimize

res = minimize(fun, x0, method='BFGS', jac=grad)
```
---

### L-BFGS-B (`method="L-BFGS-B"`)

- Упрощённый BFGS с ограниченной памятью
- Не хранит полный гессиан, а вместо этого использует несколько последних направлений
- Подходит для задач с большим числом параметров

```python
res = minimize(fun, x0, method="L-BFGS-B", jac=grad, bounds=[(0, None), (0, None)])
```

---

### Newton-CG (`method="Newton-CG"`)

- Метод Ньютона с использованием сопряжённых градиентов
- Требуется градиент и гессиан (или функция действия гессиана)

```python
res = minimize(fun, x0, method="Newton-CG", jac=grad, hess=hess)
```

---

### Сравнение методов

| Метод       | Поддержка ограничений | Нужен гессиан | Масштабируемость | Когда использовать               |
|-------------|------------------------|----------------|-------------------|----------------------------------|
| BFGS        | Нет                    | Нет            | Средняя           | Простые гладкие задачи           |
| L-BFGS-B    | Да                     | Нет            | Высокая           | Большое число параметров         |
| Newton-CG   | Нет                    | Да             | Средняя           | Есть гессиан, нужна точность     |

---

### Newton-CG (Newton conjugate gradient method)

Метод Newton-CG (Newton–Conjugate Gradient) — это модификация метода Ньютона, в которой не требуется напрямую инвертировать гессиан. Вместо этого направление \( p_k \) ищется методом сопряжённых градиентов:

$$
\nabla^2 f(x_k) \cdot p_k = -\nabla f(x_k)
$$

<p>
Это позволяет эффективно применять метод к функциям с большим числом переменных без хранения полной матрицы гессиана. Вместо неё используется функция, вычисляющая <code>H · v</code> — произведение гессиана на вектор.
</p>

<p>
Шаг <code>&alpha;<sub>k</sub></code> может подбираться внутренне (встроенным line search) или задаваться явно.
</p>

### Используется:

- Там, где гессиан трудно хранить или инвертировать

---

### **Преимущества:**

- Не требует хранения гессиана
- Подходит для задач с большим числом переменных
- Быстрее обычного Ньютона на крупных задачах

---

### **Недостатки:**

- Требует реализации действия гессиана на вектор (или приближённой функции)
- Не работает с ограничениями
- Может плохо сходиться при плохой обусловленности

Метод Newton-CG особенно полезен в задачах, где гессиан не хочется хранить, но его действие на вектор можно вычислить.


---

### Анализ поведения на функции №1

In [ ]:
def wrapped_func(x):
    val = new_func(x)
    fx = float(val)
    trajectory.append([x[0], x[1], fx])
    return fx


f_expr = x ** 2 + y ** 2 - 1
new_func = Function(f_expr)

trajectory = []

jac_func = new_func.grad_numpy_callable()

result = minimize(wrapped_func, x0=np.array([-1.56, 2.34]), method='Newton-CG', jac=jac_func)
print(f'Минимум функции: {result.fun}')
print(f'x: {result.x[0]}, y: {result.x[1]}')

d3_create_graphic(x ** 2 + y ** 2 - 1, minimum=(0, 0, -1), dataset=trajectory)
d3_create_graphic(x ** 2 + y ** 2 - 1, minimum=(0, 0, -1), dataset=trajectory, view_2d=True)

---

### Анализ поведения на функции №2

In [ ]:
def wrapped_func(x):
    val = new_func(x)
    fx = float(val)
    trajectory.append([x[0], x[1], fx])
    return fx


np_func = sympy.sin(x) * sympy.cos(y) + x ** 2 + y ** 2
new_func = Function(np_func)
trajectory = []

jac_func = new_func.grad_numpy_callable()

result = minimize(wrapped_func, x0=np.array([10, -1.2]), method='Newton-CG', jac=jac_func)
print(f'Минимум функции: {result.fun}')
print(f'x: {result.x[0]}, y: {result.x[1]}')

d3_create_graphic(
    new_func.function,
    dataset=trajectory
)

d3_create_graphic(
    new_func.function,
    dataset=trajectory,
    view_2d=True
)


---

### Квазиньютоновские методы

Квазиньютоновские методы приближают метод Ньютона, не вычисляя гессиан явно. Вместо этого на каждой итерации строится приближение к гессиану на основе градиентов:

$$
x_{k+1} = x_k + \alpha_k \cdot p_k, \quad \text{где } p_k = -B_k^{-1} \cdot \nabla f(x_k)
$$

<p>
<code>B<sub>k</sub></code> — это приближённая матрица гессиана. Она обновляется на каждом шаге, что позволяет применять метод, не зная вторых производных.
</p>

### Основные методы:

- <strong>BFGS</strong> — классический квазиньютоновский метод, хранищий полную аппроксимацию гессиана
- <strong>L-BFGS-B</strong> — «обрезанная» версия, не хранящая весь гессиан

---

### **Преимущества:**

- Не требует гессиана
- Подходит для средних и больших задач
- L-BFGS-B работает с ограничениями

---

### **Недостатки:**

- Медленнее чистого Ньютона
- Менее точен на сложных рельефах
- Требует аккуратного подбора параметров и начальной точки

Квазиньютоновские методы — разумный компромисс, если гессиан вычислять невозможно или неэффективно. Они широко применяются на практике.

---

### BFGS

### Анализ поведения на функции №1

In [ ]:
def wrapped_func(x):
    val = new_func(x)
    fx = float(val)
    trajectory.append([x[0], x[1], fx])
    return fx


f_expr = x ** 2 + y ** 2 - 1
new_func = Function(f_expr)

trajectory = []

jac_func = new_func.grad_numpy_callable()

result = minimize(wrapped_func, x0=np.array([-1.56, 2.34]), method='BFGS', jac=jac_func)
print(f'Минимум функции: {result.fun}')
print(f'x: {result.x[0]}, y: {result.x[1]}')

d3_create_graphic(x ** 2 + y ** 2 - 1, minimum=(0, 0, -1), dataset=trajectory)
d3_create_graphic(x ** 2 + y ** 2 - 1, minimum=(0, 0, -1), dataset=trajectory, view_2d=True)

---

### Анализ поведения на функции №2

In [ ]:
def wrapped_func(x):
    val = new_func(x)
    fx = float(val)
    trajectory.append([x[0], x[1], fx])
    return fx


np_func = sympy.sin(x) * sympy.cos(y) + x ** 2 + y ** 2
new_func = Function(np_func)
trajectory = []

jac_func = new_func.grad_numpy_callable()

result = minimize(wrapped_func, x0=np.array([-1.5, 1.5]), method='BFGS', jac=jac_func)
print(f'Минимум функции: {result.fun}')
print(f'x: {result.x[0]}, y: {result.x[1]}')

d3_create_graphic(
    new_func.function,
    minimum=(result.x[0], result.x[1], result.fun),
    dataset=trajectory
)

d3_create_graphic(
    new_func.function,
    minimum=(result.x[0], result.x[1], result.fun),
    dataset=trajectory,
    view_2d=True
)

---

### L-BFGS-B

### Анализ поведения на функции №1

In [ ]:
def wrapped_func(x):
    val = new_func(x)
    fx = float(val)
    trajectory.append([x[0], x[1], fx])
    return fx


f_expr = x ** 2 + y ** 2 - 1
new_func = Function(f_expr)

trajectory = []

jac_func = new_func.grad_numpy_callable()

result = minimize(wrapped_func, x0=np.array([-1.56, 2.34]), method='L-BFGS-B', jac=jac_func)
print(f'Минимум функции: {result.fun}')
print(f'x: {result.x[0]}, y: {result.x[1]}')

d3_create_graphic(x ** 2 + y ** 2 - 1, minimum=(0, 0, -1), dataset=trajectory)
d3_create_graphic(x ** 2 + y ** 2 - 1, minimum=(0, 0, -1), dataset=trajectory, view_2d=True)

---

### Анализ поведения на функции №2

In [ ]:
def wrapped_func(x):
    val = new_func(x)
    fx = float(val)
    trajectory.append([x[0], x[1], fx])
    return fx


np_func = sympy.sin(x) * sympy.cos(y) + x ** 2 + y ** 2
new_func = Function(np_func)
trajectory = []

jac_func = new_func.grad_numpy_callable()

result = minimize(wrapped_func, x0=np.array([-1.5, 1.5]), method='L-BFGS-B', jac=jac_func)
print(f'Минимум функции: {result.fun}')
print(f'x: {result.x[0]}, y: {result.x[1]}')

d3_create_graphic(
    new_func.function,
    minimum=(result.x[0], result.x[1], result.fun),
    dataset=trajectory
)

d3_create_graphic(
    new_func.function,
    minimum=(result.x[0], result.x[1], result.fun),
    dataset=trajectory,
    view_2d=True
)

---

## Реализация квазиньютоновского метода

### Квазиньютоновский метод (BFGS)

Одним из наиболее популярных методов является метод Бройдена-Флетчера-Гольдфарба-Шанно (BFGS). Он сохраняет быструю сходимость метода Ньютона, но при этом более устойчив и применим, когда гессиан недоступен или дорог в вычислении. Собственно говоря, его мы и реализовали

Метод BFGS использует следующую схему:

- считаем градиент в текущей точке
- движемся по направлению $p_k = -H_k \nabla f(x_k)$
- обновляем $H_k$ на основе изменений $x$ и градиента:
  
  $$
  H_{k+1} = (I - \rho_k s_k y_k^T) H_k (I - \rho_k y_k s_k^T) + \rho_k s_k s_k^T
  $$

где $s = x_{k+1} - x_k$, $y = \nabla f_{k+1} - \nabla f_k$, $\rho = 1 / (y_k^T s_k)$.

#### Реализация

- `grad_func` и `func` — обёртки над `sympy`, возвращающие `numpy`-массивы
- `H_k` — приближение к обратному Гессиану, инициализируется как единичная матрица
- шаг `alpha` фиксированный
- сохраняется траектория, используется для отрисовки графика

При запуске с `.set_create_graphic()` автоматически рисуется график и траектория.

#### Пример

```python
BackTrace(func).set_bfgs(alpha=1.0).set_create_graphic().start_back_trace()
```
---


In [ ]:
func = Function(sympy.sin(x) * sympy.cos(y) + x ** 2 + y ** 2)
BackTrace(func).set_bfgs(alpha=1.0).set_create_graphic().start_back_trace(start_point=[1.54, -1.59])

---

## Оптимизация гиперпараметров с помощью optuna

### Проверим лучший константный шаг

In [ ]:
from source.optima_func.path_to_optima import objective_constant_step
import time
import optuna
from sympy import symbols

study = optuna.create_study(directions=["minimize", "minimize"])
study.optimize(
    lambda trial: objective_constant_step(trial, create_func=Function(x ** 2 + y ** 2), start_point=[100, 100]),
    n_trials=50)

best = min(
    study.best_trials,
    key=lambda t: t.values[0] + 0.05 * t.values[1]
)

print("Лучшая сбалансированная точка:")
print("Параметры:", best.params)
print("Функция:", best.values[0], "Время:", best.values[1])



---

### Проверим границы для дихотомии

In [ ]:

from source.optima_func.path_to_optima import objective_dichotomy_step

study = optuna.create_study(directions=["minimize", "minimize"])
study.optimize(
    lambda trial: objective_dichotomy_step(trial, create_func=Function(x ** 2 + y ** 2), start_point=[100, 100]),
    n_trials=50)

best = min(study.best_trials, key=lambda t: t.values[0] + 0.05 * t.values[1])  # точность + штраф за время

print("Параметры лучшего trial-а:")
print("  left_border:", best.params["left_border"])
print("  right_border:", best.params["right_border"])
print("Функция:", best.values[0], "Время:", best.values[1])

---

## Сравним BackTrace из ЛР1 и текущую реализацию

Решая прошлую лабораторную работу, мы заметили, что градиентный спуск плохо работает для очень быстрорастущих функций. Например рассмотрим:

 $$ z = x^{10} + y^2 $$

При запуске на какой-то далёкой от нуля точки, например на [500,500],

значение нашей функции > 976 562 500 000 000 000 000 000 000, что в общем то не мало. После первой попытки пойти по антиградиенту (а это тоже ничего себе функция), мы попадём в ещё более далёкую точку, с ещё более большим значением функции. Туши свет.

| Iterations | Step |     y     | x             | norma        | F                     |
|------------|------|-----------|---------------|--------------|-----------------------|
|     1      |  1   |  -500.0   | -1.953125e+25 | 1.953125e+25 | 8.07793566946316e+252 |
|     2      |  1   |   500.0   | ...           | ....         | .....

```OverflowError: (34, 'Result too large')```

Посмотрим как с этой проблемой справится наш улучшенный спуск


In [ ]:
BackTrace(Function(x**10 + y**2)).set_piecewise_constant().set_create_graphic().start_back_trace()

Как можно заметить, мы победили ^^

---
Ужесточим ситуацию

In [ ]:
BackTrace(Function(x**30 + y**30)).set_piecewise_constant().set_create_graphic().start_back_trace(start_point=[778, 46])

---
Однако если мы запустим на не очень приятной функции, то тоже упадём, но уже на вычислении гессиана

In [ ]:
import traceback
from sympy.core.evalf import PrecisionExhausted
try:
    BackTrace(Function(x**20 + y**13)).set_piecewise_constant().set_create_graphic().start_back_trace(start_point=[333, 927])
except PrecisionExhausted as ex:
    traceback.print_exc()

